In [14]:
#................................................INSTALLING THE REQUIRED DEPENDENCIES FOR ML PROJECT.........................................

In [1]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import numpy as np
import re  #regular expression :: Important for searching in text data
from nltk.corpus import stopwords  #that is import popular stopwords from natural language toolkit. Corpus stands for body of the file or here toolkit
from nltk.stem.porter import PorterStemmer  #used for generating the root word after stemming the word from nlt
from sklearn.feature_extraction.text import TfidfVectorizer  #used to convert the given words into required feature vector
from sklearn.model_selection import train_test_split  #to split the data into training and testing data
from sklearn.linear_model import LogisticRegression  #will use logistic regression as it is best for binary classification problem as ours
from sklearn.metrics import accuracy_score #for performance evaluation

In [11]:
import nltk
nltk.download('stopwords')
#getting all the stopwords

[nltk_data] Downloading package stopwords to /Users/rajveer-
[nltk_data]     mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
#showing all stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [15]:
#.......................................................DATA PREPROCESSING STEP.....................................................................

In [16]:
#loading dataset to Pandas Dataframe
news_dataset = pd.read_csv('/Users/rajveer-mac/Desktop/ML Project/fake-news/train.csv')

In [19]:
news_dataset.shape

(20800, 5)

In [20]:
#hence we have 20800 rows that is these many news articles and 5 cols/features for each

In [21]:
#printing first 5 rows of dataset
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [22]:
#counting the number of missing values in the dataset in each column or feature
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [23]:
#as our dataset is large compared to number of missing values hence we will drop these missing values. Else we would had to immune them by providing some values to them
#replacing null values with empty string = ''
news_dataset = news_dataset.fillna('')

In [24]:
# we will perform prediction using Title and author data only. As the text data could be very large and will take more time complexity

In [25]:
#merging the author name and news title
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title'] 

In [26]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [29]:
#separating the data except label in one variable and label column in another for regression function
X = news_dataset.drop(columns = 'label', axis = 1) # i.e drop the label column
Y = news_dataset['label']
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [ ]:
#..........................................................STEMMING......................................................................

In [64]:
#example :: actor, acting, actress ----> act

In [31]:
port_stem = PorterStemmer()

In [34]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content) # considering pure english text{exclude numbers, symbols} from given string = content
                                                        # the above line will replace all the characters not in a-z, A-Z with space
    
    stemmed_content = stemmed_content.lower()  # convert all the letters to lowercase as ML model might think that words with uppercase would be important
    
    stemmed_content = stemmed_content.split() # converting all the words to list
    
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] # performing stemming for each word {excluding stop words}
    stemmed_content = ' '.join(stemmed_content) #finally joining all the words from list with space to from string back
    
    return stemmed_content

In [36]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [37]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [38]:
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [39]:
print(X)
print(Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [40]:
#.........................................................TEXT TO NUMBER......................................................................

In [41]:
#converting the textual data to numerical data
vectorizer = TfidfVectorizer()  # Tf(term frequency) This counts the number of times a significant word appers in text . idf(inverse document frequency) = determines a word is significant or not 
vectorizer.fit(X)
X = vectorizer.transform(X) #convert the values to features

In [42]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.4154496266472162
  (2, 9620)	0.4935149294364995
  (2, 5968)	0.34746133867282925
  (2, 5389)	0.38665305511826153
  (2, 3103)	0.4609748958322965
  :	:
  (20797, 13122)	0.24825263521976057
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396505
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.17455348025522197
  (20797, 9518)	0.2954

In [43]:
#.........................................................SPLITTING DATA......................................................................

In [45]:
#splitting data into training and testing
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, stratify = Y, random_state = 2) # we assgin 80% data as training and 20% as testing

#we use stratify so that there is same proportion of Y in testing data as it is in whole data. i.e agar y = 1 for 90% cases in whole data then in testing data it Y=1 for 90% of caeses with the help of stratify

In [46]:
#.........................................................MODEL TRAIING :: LOGISTIC REGRESSION MODEL......................................................................

In [47]:
model = LogisticRegression()

In [48]:
model.fit(X_train, Y_train)

LogisticRegression()

In [50]:
#.................................................MODEL EVALUATION......................................................................

In [51]:
# accuracy score on the taining data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [54]:
print('Accuracy score of training data : ', training_data_accuracy*100, '%')

Accuracy score of traiing data :  98.6358173076923 %


In [55]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [56]:
print('Accuracy score of testing data : ', test_data_accuracy*100)

Accuracy score of testing data :  97.90865384615385


In [57]:
#.................................................MAKING PREDICTION SYSTEM......................................................................

In [62]:
X_new = X_test[0]
prediction = model.predict(X_new)
print(prediction)

if(prediction[0] == 0):
    print('THE NEWS IS FAKE!!!')
else:
    print('THE NEWS IS REAL.')

print(Y_test[0])

[1]
THE NEWS IS REAL.
1


In [63]:
X_new = X_test[1]
prediction = model.predict(X_new)
print(prediction)

if(prediction[0] == 0):
    print('THE NEWS IS FAKE!!!')
else:
    print('THE NEWS IS REAL.')

print(Y_test[1])

[0]
THE NEWS IS FAKE!!!
0
